## Problem 1

Find the absolute minimum of the function

\begin{equation}
    f(x) = f(x_1, x_2) = x_1 e^{-{x_1}^2 -{x_2}^2}
\end{equation}

in the domain $x \in \mathbb{R}^2$ (unconstrained problem).

The initial point is $x^0 = (-0.6, -0.3)$.

In [12]:
## Levenberg-Marquardt algorithm (2nd order)

import sympy as sp
import numpy as np
import pandas as pd
from sympy.utilities.lambdify import lambdify
from scipy.optimize import minimize_scalar

def LevenbergMarquardt(f, x0, ε, J):
    # Define symbolic variables for optimization
    x1, x2 = sp.symbols('x1 x2')

    # Calculate the gradient (first derivative) of the objective function
    grad_f = sp.Matrix([sp.diff(f, x1), sp.diff(f, x2)])

    # Calculate the Hessian matrix (second derivative) of the objective function
    hessian_f = sp.Matrix([[sp.diff(grad_f[i], x1), sp.diff(grad_f[i], x2)] for i in range(2)])

    # Create lambdified functions for numerical computation
    f_lambda = lambdify((x1, x2), f, 'numpy')
    grad_lambda = lambdify((x1, x2), grad_f, 'numpy')
    hessian_lambda = lambdify((x1, x2), hessian_f, 'numpy')

    # Initialize x with the initial point
    x = np.array(x0, dtype=float)
    num_iterations = 0

    # The starting β is one
    β = 1

    # Create an empty list to store optimization data
    optimization_data = []

    λ = None  # Initialize λ

    while num_iterations <= J:
        g = np.array(grad_lambda(x[0], x[1]), dtype=float)
        H = np.array(hessian_lambda(x[0], x[1]), dtype=float)
        s = np.reshape(-np.linalg.inv((H + np.array(β*np.eye(2), dtype=float))) @ g, x.shape)
        grad_norm = np.linalg.norm(g)
        fx = f_lambda(x[0], x[1])

        if grad_norm <= ε or num_iterations == J:
            break

        # Define a function for g(λ) that takes λ as an argument and returns f(x + λs)
        g_lambda = lambda lam: f_lambda(x[0] - lam * s[0], x[1] - lam * s[1])

        # Use minimize_scalar to find the optimal λ
        result = minimize_scalar(g_lambda)
        λ = float(result.x)

        optimization_data.append({'Iteration': num_iterations, 'x1': x[0], 'x2': x[1], 'f(x)': fx, 'Gradient Norm': grad_norm, 'λ': λ, 'β': β})

        prev = f_lambda(x[0], x[1])
        x_next = x + λ*s
        next = f_lambda(x_next[0], x_next[1])

        if next < prev:
            # Only update x if the new estimate improves the objective function
            β = β*2
        else:
            β = β/4

        x = x_next


        num_iterations += 1

    # Convert the list of dictionaries to a DataFrame
    optimization_data_df = pd.DataFrame(optimization_data)

    print("Iterations performed:", num_iterations)
    print("The point that minimizes the function f is [" + str(x[0]) + ', ' + str(x[1]) + '], such that f(x) = ' + str(fx))

    return fx, optimization_data_df

# Define your objective function correctly using 'x1_val' and 'x2_val'
x1, x2 = sp.symbols('x1 x2')
def f(x1_val, x2_val):
    return x1_val * sp.exp(-x1_val**2 - x2_val**2)

tolerance = 1e-15
initial_point = [-0.4, -0.01]

# Call the Newton's method with λ and β optimization
result, optimization_data = LevenbergMarquardt(f(x1, x2), initial_point, tolerance, 100)

# Now the 'optimization_data' DataFrame includes the 0th iteration and f(x) value.
print(optimization_data)


Iterations performed: 4
The point that minimizes the function f is [-42.96138907704416, -24.101290136910723], such that f(x) = -0.0
   Iteration        x1       x2      f(x)  Gradient Norm           λ       β
0          0 -0.400000 -0.01000 -0.340823       0.579440   -1.498286  1.0000
1          1 -0.092869 -0.01819 -0.092041       0.973995   -0.500680  0.2500
2          2  0.519800 -0.07208  0.394673       0.353583    5.808796  0.0625
3          3  1.613004  0.46434  0.096396       0.266685 -120.373839  0.1250
